## libaray import

In [164]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## preprocess

In [165]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [166]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [167]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [168]:
# timestamp 삭제
ratings.drop('timestamp', axis=1, inplace=True)
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [169]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 분석

In [170]:
# ratings에 있는 유니크한 영화 개수
new_ratings['title'].nunique()

3628

In [171]:
# rating에 있는 유니크한 사용자 수
new_ratings['user_id'].nunique()

6040

In [172]:
# 가장 인기 있는 영화 30개(인기순)
# ratings에 있는 유니크한 영화 개수
num_movie = ratings.movie_id.nunique()
# ratings에 있는 유니크한 사용자 수
num_user = ratings.user_id.nunique()

merge_df = pd.merge(ratings, movies, on='movie_id', how='left')
popular = merge_df.groupby('movie_id')['count'].sum().to_frame().reset_index()
popular = pd.merge(popular, movies, on='movie_id', how='left')

popular.sort_values(by='count', ascending=False)[:30]

,movie_id,count,title,genre
2600,2858,14449,American Beauty (1999),Comedy|Drama
249,260,13178,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1080,1196,12648,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1810,2028,11348,Saving Private Ryan (1998),Action|Drama|War
1094,1210,11303,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1082,1198,11179,Raiders of the Lost Ark (1981),Action|Adventure
573,593,11096,"Silence of the Lambs, The (1991)",Drama|Thriller
2325,2571,10903,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2507,2762,10703,"Sixth Sense, The (1999)",Thriller
569,589,10513,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller


In [173]:
# title 로 movie_id 찾기
def findId(title):
    id = movies.loc[movies['title']==title, 'movie_id'].values
    return id[0]

In [174]:
# movie_id 로 title 찾기
def findTitle(id):
    title = movies.loc[movies['movie_id']==id, 'title'].values
    return title[0]

### 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [175]:
# 내가 좋아하는 영화를 추가하기 위해 영화 데이터베이스에서 찾기

# 가장 인기있는 70개 중에서 선택
popular_movies.sort_values(ascending=False).head(70)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
                                                         ... 
Big (1988)                                               1386
Butch Cassidy and the Sundance Kid (1969)                1386
Twelve Monkeys (1995)                                    1384
Close Encounters of the Third Kind (1977)                1374
Amadeus (1984)                                           1340
Name: user_id, Length: 70, dtype: int64

In [176]:
my_favorite = ['Jurassic Park (1993)' , 'Terminator 2: Judgment Day (1991)' ,'Forrest Gump (1994)' ,'Die Hard (1988)' ,'Mission: Impossible (1996)']
my_favorite_id = []
for e in my_favorite:
    my_favorite_id.append(findId(e))
my_favorite_id

[480, 589, 356, 1036, 648]

In [178]:
my_playlist = pd.DataFrame({'user_id': ['me']*5, 'movie_id': my_favorite_id, 'count':[5]*5})
if not ratings.isin({'user_id':['me']})['user_id'].any():  
    ratings = ratings.append(my_playlist) 

ratings.reset_index(inplace=True, drop=True)
ratings.tail(10)

,user_id,movie_id,count
836473,6040,1090,3
836474,6040,1094,5
836475,6040,562,5
836476,6040,1096,4
836477,6040,1097,4
836478,me,480,5
836479,me,589,5
836480,me,356,5
836481,me,1036,5
836482,me,648,5


In [179]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [182]:
# 인덱싱 확인
print(user_to_idx['me'])    # 6038명의 유저 중 마지막으로 추가된 유저이니 6039이여아 한다 
print(movie_to_idx[findId('Jurassic Park (1993)')])

6039
107


In [183]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):
    print('user_id indexing succes')
    ratings['user_id'] = temp_user_data
else:
    print('user_id indexing fail')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie indexing succes')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie indexing fail')

user_id indexing succes
movie indexing succes


### CSR matrix를 직접 만들어 봅시다

In [184]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [185]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [186]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=5, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/5 [00:00<?, ?it/s]

### 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [196]:
# 내가 선호하는 영화
me, jurassic_park = user_to_idx['me'], movie_to_idx[findId('Jurassic Park (1993)')]
me_vector, jurassic_park_vector = als_model.user_factors[me], als_model.item_factors[jurassic_park]

# me과 jurassic_park를 내적
print(np.dot(me_vector, jurassic_park_vector))

0.6767865


In [197]:
# 내가 선호하지 않는 영화
me, american_beauty = user_to_idx['me'], movie_to_idx[findId('American Beauty (1999)')]
me_vector, american_beauty_vector = als_model.user_factors[me], als_model.item_factors[american_beauty]

# me과 american_beauty를 내적
print(np.dot(me_vector, american_beauty_vector))

0.064954974


### 내가 좋아하는 영화와 비슷한 영화를 추천

In [200]:
favorite_movie_id = findId('Jurassic Park (1993)')
movie_id = movie_to_idx[favorite_movie_id]
similar_movie_id = als_model.similar_items(movie_id, N=15)
similar_movie = []
for e in similar_movie_id:
    similar_movie.append((findTitle(e[0]), e[1]))
similar_movie 

[('Muppet Treasure Island (1996)', 1.0),
 ('Kids (1995)', 0.83986074),
 ('Mary Reilly (1996)', 0.7586105),
 ("Mr. Holland's Opus (1995)", 0.66902196),
 ("Antonia's Line (Antonia) (1995)", 0.60271215),
 ("Star Maker, The (Uomo delle stelle, L') (1995)", 0.57837313),
 ('Postino, Il (The Postman) (1994)', 0.57640284),
 ('Bad Boys (1995)', 0.5022491),
 ('Birdcage, The (1996)', 0.49437332),
 ('Neon Bible, The (1995)', 0.49049452),
 ('Hudsucker Proxy, The (1994)', 0.47617492),
 ('Dunston Checks In (1996)', 0.4745544),
 ('Judgment Night (1993)', 0.47270435),
 ('Big Bully (1996)', 0.47044325),
 ('Striking Distance (1993)', 0.4695934)]

### 내가 가장 좋아할 만한 영화들을 추천

In [201]:
user = user_to_idx['me']

movie_recommended_id = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended = []
for e in movie_recommended_id:
    movie_recommended.append((findTitle(e[0]), e[1]))
movie_recommended

[('Kids (1995)', 0.46823898),
 ("Mr. Holland's Opus (1995)", 0.40197235),
 ('Birdcage, The (1996)', 0.4011376),
 ("Star Maker, The (Uomo delle stelle, L') (1995)", 0.36892685),
 ('Tie That Binds, The (1995)', 0.36882907),
 ('Big Bully (1996)', 0.34779865),
 ("Antonia's Line (Antonia) (1995)", 0.2937768),
 ('Race the Sun (1996)', 0.286111),
 ('Johnny Mnemonic (1995)', 0.25616395),
 ('Dunston Checks In (1996)', 0.255159),
 ('French Kiss (1995)', 0.24417776),
 ('Copycat (1995)', 0.23878554),
 ('Mortal Kombat (1995)', 0.23864885),
 ('Cutthroat Island (1995)', 0.23755932),
 ('Two if by Sea (1996)', 0.23381609),
 ('It Takes Two (1995)', 0.22977525),
 ('Pocahontas (1995)', 0.21887803),
 ("Young Poisoner's Handbook, The (1995)", 0.20432085),
 ('Indian in the Cupboard, The (1995)', 0.19113736),
 ('Underground (1995)', 0.19103009)]

## 회고
- 이번 프로젝트에서 어려웠던 점
    - 추천 모델을 처음 접하게 되면서 CSR의 특징을 파악하는데 많은 시간이 걸렸다.
- 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점
    - 어느 정도 이해는 됐지만, 추천 기능을 업그레이드하는데는 어려움이 많을 것 같다.
- 자기 다짐
    - 새로운 것을 계속 배우면서 점점 쌓는 방식으로 해야겠다.